# Main OpenFOAM Notebook

In [ ]:
import numpy as np
import os
import subprocess

# Definisci i range di angolo d'attacco e velocità da testare
attack_angles = np.linspace(-10, 10, 21)
velocities = np.linspace(10, 50, 11)

# Crea il dataset vuoto
dataset = []

# Esegui le simulazioni OpenFOAM
for angle in attack_angles:
    for velocity in velocities:
        # Genera la geometria
        os.system(f'blockMesh -case case_{angle}_{velocity}')
        
        # Imposta i parametri di simulazione
        with open('system/controlDict', 'w') as f:
            f.write(f'attackAngle {angle};\nvelocity {velocity};')
        
        # Esegui la simulazione
        subprocess.run(['simpleFoam', '-case', f'case_{angle}_{velocity}'])
        
        # Estrai i risultati
        with open(f'case_{angle}_{velocity}/forces.dat', 'r') as f:
            cl, cd = [float(x) for x in f.readlines()[-1].split()]
        
        # Aggiungi i dati al dataset
        dataset.append([angle, velocity, cl, cd])

# Salva il dataset in un file
np.savetxt('dataset.csv', dataset, delimiter=',', header='angle,velocity,cl,cd')